In [ ]:
%pylab inline --no-import-all
#%pylab notebook --no-import-all
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = [24, 8]

In [ ]:
names = 'time commit engine case solverTime chainTime wholeTime partialBrCov fullBrCov blockCov pathNum brQueryNum testQueryNum cexCacheHit'
df = pd.read_csv('bench.csv', header=0, names=names.split(), parse_dates=['time'])
df = df.convert_dtypes(False, True, False, False, False)
df

In [ ]:
tmp = df['case'].str.extract(r'(?:(par\d+)_)?(.+)')
df['para'] = tmp[0].fillna('serial')
df['case'] = tmp[1]

In [ ]:
df['commit'] = pd.Categorical(df['commit'], df['commit'].unique(), ordered=True)
df['para'] = pd.Categorical(df['para'], ['serial', 'par2', 'par4', 'par8', 'par16'], ordered=True)
df['engine'] = pd.Categorical(df['engine'], ['ImpLLSC', 'ImpCPSLLSC', 'PureLLSC', 'PureCPSLLSC', 'PureCPSLLSC_Z3'], ordered=True)

In [ ]:
df = df[df['commit'] >= df['commit'].dtype.categories[-12:][0]].copy()
df['commit'] = df['commit'].cat.remove_unused_categories()

In [ ]:
vio_styles = dict(inner=None, scale='count')
for name, case in df.groupby('case'):
    fig, axes = plt.subplots(1, 3, sharey=True)

    whole = case.pivot_table(index='commit', columns=['engine', 'para'], values='wholeTime', aggfunc='mean')
    werr = case.pivot_table(index='commit', columns=['engine', 'para'], values='wholeTime', aggfunc='std')
    ax = whole.plot(ax=axes[0], yerr=werr)
    ax.set_title(name)
    
    data = case[case['para'] == "serial"]
    ax = sns.violinplot(x='commit', y='wholeTime', hue='engine', data=data, ax=axes[1], **vio_styles)
    ax.set_title('%s - serial' % name)
    
    data = case[case['engine'] == "PureCPSLLSC_Z3"]
    ax = sns.violinplot(x='commit', y='wholeTime', hue='para', data=data, ax=axes[2], **vio_styles)
    ax.set_title('%s - parallel' % name)

In [ ]:
df = df[df['commit'] >= df['commit'].dtype.categories[-6:][0]].copy()
df['commit'] = df['commit'].cat.remove_unused_categories()
df['threads'] = df['para'].str.extract(r'par(\d+)').fillna(1).astype(int)
df['cacheTime'] = (df['chainTime'] - df['solverTime']) / df['threads']
df['stateTime'] = df['wholeTime'] - df['chainTime'] / df['threads']
df['solverTime'] /= df['threads']
#pd.options.display.max_rows = None

In [ ]:
serial = df[~df['engine'].str.endswith('_Z3')]
serial.pivot_table(index=['case', 'commit'], columns=['engine'], values=['solverTime', 'cacheTime', 'stateTime'], aggfunc='median')

In [ ]:
para = df[df['engine'].str.endswith('_Z3')]
para.pivot_table(index=['case', 'commit'], columns=['para'], values=['solverTime', 'cacheTime', 'stateTime'], aggfunc='median')